In [79]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

options = Options()
options.headless = True

import numpy as np
import pandas as pd
import time as time
from time import sleep
import random
from tqdm import tqdm


In [13]:
from nba_api.stats.endpoints import leaguegamelog
from nba_api.stats.library.parameters import SeasonAll

In [51]:
seasons = ['20{}-{}'.format(x, x+1) for x in range(10, 21)]
dates = []
for season in seasons:
    date = []
    for season_type in ['Regular Season', 'Playoffs']:
        games = leaguegamelog.LeagueGameLog(season=season).get_data_frames()[0]
        dates.extend(games['GAME_DATE'].unique())

dates[-10:]

['2021-02-19',
 '2021-02-20',
 '2021-02-21',
 '2021-02-22',
 '2021-02-23',
 '2021-02-24',
 '2021-02-25',
 '2021-02-26',
 '2021-02-27',
 '2021-02-28']

In [52]:
print(len(dates))

3298


In [84]:
# Get Moneylines

gm_date = []
away_teams = []
home_teams = []
away_scoreboards = []
home_scoreboards = []
away_moneylines = []
home_moneylines = []

for date in tqdm(dates[109:], desc='progress'):
    web = 'https://www.sportsbookreview.com/betting-odds/nba-basketball/money-line/?date={}'.format(date)
    path = '../chromedriver.exe'
    driver = webdriver.Chrome(path)
    driver.get(web)

    single_row_events = driver.find_elements_by_class_name('eventMarketGridContainer-3QipG')

    num_postponed_events = len(driver.find_elements_by_class_name('eventStatus-3EHqw'))

    num_listed_events = len(single_row_events)
    cutoff = num_listed_events - num_postponed_events

    for event in single_row_events[:cutoff]:

        away_team = event.find_elements_by_class_name('participantBox-3ar9Y')[0].text
        home_team = event.find_elements_by_class_name('participantBox-3ar9Y')[1].text
        away_teams.append(away_team)
        home_teams.append(home_team)
        gm_date.append(date)


        scoreboard = event.find_elements_by_class_name('scoreboard-1TXQV')

        home_score = []
        away_score = []

        for score in scoreboard:
            quarters = score.find_elements_by_class_name('scoreboardColumn-2OtpR')
            for i in range(len(quarters)):
                scores = quarters[i].text.split('\n')
                away_score.append(scores[0])
                home_score.append(scores[1])
            away_scoreboards.append(away_score)
            home_scoreboards.append(home_score)

        mls = event.find_elements_by_class_name('pointer-2j4Dk')
        away_lines = []
        home_lines = []
        for i in range(len(mls)):    
            if i % 2 == 0:
                away_lines.append(mls[i].text)
            else:
                home_lines.append(mls[i].text)
        away_moneylines.append(away_lines)
        home_moneylines.append(home_lines)
        
    driver.quit()
    sleep(random.randint(3,5))



progress: 100%|█████████████████████████████████████████████████████████████████| 3189/3189 [12:25:20<00:00, 14.02s/it]


In [86]:
print(len(away_teams))
print(len(home_teams))
print(len(gm_date))
print(len(away_scoreboards))
print(len(home_scoreboards))
print(len(away_moneylines))
print(len(home_moneylines))

df1 = pd.read_csv('../data/moneylines.csv')

df2 = pd.DataFrame({'away_team':away_teams[:-90],
                  'home_team':home_teams[:-90],
                   'game_date':gm_date[:-90],
                  'away_scoreboard':away_scoreboards[:-90],
                  'home_scoreboard':home_scoreboards[:-90],
                  'away_moneyline':away_moneylines[:-90],
                  'home_moneyline':home_moneylines[:-90]})




23858
23858
23858
23768
23768
23858
23858


In [89]:
df1 = df1.drop(columns=['away_scoreboard', 'home_scoreboard'])

In [91]:
all_moneylines = pd.concat([df1, df2])
all_moneylines.to_csv('../data/all_moneylines.csv', index=False)


In [94]:
set(dates) - set(all_moneylines['game_date'])

{'2011-02-12', '2011-02-15', '2011-02-16'}

In [75]:
dates[108]

'2011-02-12'

In [76]:
df.tail(20)

,away_team,home_team,game_date,away_scoreboard,home_scoreboard,away_moneyline,home_moneyline
771,Detroit,Cleveland,2011-02-09,"[25, 30, 26, 22, 103]","[16, 27, 27, 24, 94]","[-, -, +105, -]","[-, -, -125, -]"
772,San Antonio,Toronto,2011-02-09,"[31, 29, 21, 30, 111]","[29, 27, 28, 16, 100]","[-400, -400, -365, -]","[+300, +300, +305, -]"
773,Charlotte,Indiana,2011-02-09,"[28, 20, 22, 33, 103]","[29, 29, 23, 23, 104]","[-, +160, -, -]","[-, -190, -, -]"
774,L.A. Clippers,New York,2011-02-09,"[28, 30, 30, 28, 116]","[29, 18, 31, 30, 108]","[-, +280, +305, -]","[-, -350, -365, -]"
775,Chicago,Utah,2011-02-09,"[21, 23, 22, 25, 91]","[23, 18, 19, 26, 86]","[-, -, -, -]","[-, -, -, -]"
776,Dallas,Sacramento,2011-02-09,"[20, 36, 18, 28, 102]","[28, 19, 26, 27, 100]","[-230, -200, -215, -]","[+190, +170, +185, -]"
777,Denver,Golden State,2011-02-09,"[35, 33, 24, 22, 114]","[34, 36, 30, 16, 116]","[-, +130, +130, -]","[-, -150, -150, -]"
778,L.A. Lakers,Boston,2011-02-10,"[20, 25, 27, 20, 92]","[27, 26, 15, 18, 86]","[-, +110, -, -]","[-, -130, -, -]"
779,Golden State,Phoenix,2011-02-10,"[22, 20, 19, 27, 88]","[32, 33, 22, 25, 112]","[-, +230, +240, -]","[-, -280, -290, -]"
780,Dallas,Denver,2011-02-10,"[19, 39, 34, 28, 120]","[33, 26, 29, 33, 121]","[-, +125, +130, -]","[-, -145, -150, -]"


In [82]:
gm_date = []
away_teams = []
home_teams = []
away_scoreboards = []
home_scoreboards = []
away_moneylines = []
home_moneylines = []

web = 'https://www.sportsbookreview.com/betting-odds/nba-basketball/money-line/?date=2011-02-13'
path = '../chromedriver.exe'
driver = webdriver.Chrome(path)
driver.get(web)

single_row_events = driver.find_elements_by_class_name('eventMarketGridContainer-3QipG')

num_postponed_events = len(driver.find_elements_by_class_name('eventStatus-3EHqw'))

num_listed_events = len(single_row_events)
cutoff = num_listed_events - num_postponed_events

for event in single_row_events[:cutoff]:

    away_team = event.find_elements_by_class_name('participantBox-3ar9Y')[0].text
    home_team = event.find_elements_by_class_name('participantBox-3ar9Y')[1].text
    away_teams.append(away_team)
    home_teams.append(home_team)
    gm_date.append(date)


    scoreboard = event.find_elements_by_class_name('scoreboard-1TXQV')

    home_score = []
    away_score = []

    for score in scoreboard:
        quarters = score.find_elements_by_class_name('scoreboardColumn-2OtpR')
        for i in range(len(quarters)):
            scores = quarters[i].text.split('\n')
            away_score.append(scores[0])
            home_score.append(scores[1])

        away_scoreboards.append(away_score)
        home_scoreboards.append(home_score)

    mls = event.find_elements_by_class_name('pointer-2j4Dk')
    away_lines = []
    home_lines = []
    for i in range(len(mls)):    
        if i % 2 == 0:
            away_lines.append(mls[i].text)
        else:
            home_lines.append(mls[i].text)
    away_moneylines.append(away_lines)
    home_moneylines.append(home_lines)

driver.quit()